In [1]:
import pandas as pd
import numpy as np

from src.language_processing2 import processing_text
from src.language_processing2 import vectorize_text
from src.language_processing2 import vectorize_desc
from src.modeling import test_model
from src.modeling import test_model2
from src.modeling import test_model3
from src.modeling import test_model5

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split,KFold

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)

In [3]:
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,1000)

In [4]:
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,1000)

In [5]:
df.head()

,text,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,spam_own,Docusign,onespan,...,desc-TF-IDF-990,desc-TF-IDF-991,desc-TF-IDF-992,desc-TF-IDF-993,desc-TF-IDF-994,desc-TF-IDF-995,desc-TF-IDF-996,desc-TF-IDF-997,desc-TF-IDF-998,desc-TF-IDF-999
0,just closed a deal in 27 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,just closed a deal in 2 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,just closed a deal in 2 hours using #AdobeSign...,0,0,0,0,1,0,0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,just closed a deal in 26 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,just closed a deal in 6 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
test_model3(df,'spam_hijack',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9201875000000002

Log-Loss score of this model is

0.25771578635491316

AUC score of this model is

0.8878695993945074



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9156244843571472

Log-Loss score of this model is

0.23974509975605765

AUC score of this model is

0.8888592257717483


In [7]:
df2 = df.sample(frac=1)
df2_training = df2[:16000]
df2_testing = df2[16000:]
y = df2_training['spam_marketing']
X = df2_training[['Docusign', 'onespan', 'signnow','adobe sign','listed_count', 'statuses_count','followers_count','favourites_count', 'friends_count','time_float_sin','time_float_cos', 'is_description_none']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
model = GradientBoostingClassifier(subsample=0.5, learning_rate=0.01)
kf = KFold(n_splits=5, shuffle=True)

ll_performance = []
auc_performance = []
acc_performance = []
#kfold split on X_ (which is X_train of len 110)
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    y_pred = model.predict_proba(X_test)
    log_ll = log_loss(y_test, y_pred)
    ll_performance.append(log_ll)
    auc = roc_auc_score(y_test,y_predict)
    auc_performance.append(auc)
    acc = accuracy_score(y_test,y_predict)
    acc_performance.append(acc)
        
        
print("\n" + "Score summary for initial test (first 80% of data)" + "\n")
print("\n" + "Accuracy score of this model is" + "\n")
print(np.mean(acc_performance))
print("\n" + "Log-Loss score of this model is" + "\n")
print(np.mean(ll_performance))
print("\n" + "AUC score of this model is" + "\n")
print(np.mean(auc_performance))
print("\n")

y = df2_testing['spam_marketing']
X = df2_testing[['Docusign', 'onespan', 'signnow','adobe sign','listed_count', 'statuses_count','followers_count','favourites_count', 'friends_count','time_float_sin','time_float_cos', 'is_description_none']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    #model = model()
kf = KFold(n_splits=5, shuffle=True)

ll_performance = []
auc_performance = []
acc_performance = []
#kfold split on X_ (which is X_train of len 110)
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    y_pred = model.predict_proba(X_test)
    log_ll = log_loss(y_test, y_pred)
    ll_performance.append(log_ll)
    auc = roc_auc_score(y_test,y_predict)
    auc_performance.append(auc)
    acc = accuracy_score(y_test,y_predict)
    acc_performance.append(acc)
        
        
print("\n" + "Score summary for final test (last 20% of data)" + "\n")
print("\n" + "Accuracy score of this model is" + "\n")
print(np.mean(acc_performance))
print("\n" + "Log-Loss score of this model is" + "\n")
print(np.mean(ll_performance))
print("\n" + "AUC score of this model is" + "\n")
print(np.mean(auc_performance))


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9410624999999999

Log-Loss score of this model is

0.17828652418686455

AUC score of this model is

0.5015183060906852



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9390929667093191

Log-Loss score of this model is

0.1921013788430044

AUC score of this model is

0.5


In [3]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,1000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,1000)
test_model5(df,'spam_marketing',col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9413125000000001

Log-Loss score of this model is

0.16940142876448258

AUC score of this model is

0.5042379474921217



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9410873565150795

Log-Loss score of this model is

0.17536699963746233

AUC score of this model is

0.5111998870694523


In [4]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,2000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,1000)
test_model5(df,'spam_marketing',col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9411875000000001

Log-Loss score of this model is

0.16979797238451652

AUC score of this model is

0.5084096996533419



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9425829932036326

Log-Loss score of this model is

0.17346175219325646

AUC score of this model is

0.5018181818181817


In [4]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,1000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,2000)
test_model3(df,'spam_marketing',GradientBoostingClassifier,col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9523125

Log-Loss score of this model is

0.12953006112247994

AUC score of this model is

0.6397430666305363



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9513242486791761

Log-Loss score of this model is

0.14101985031141395

AUC score of this model is

0.6265075664720406


In [6]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,2000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,2000)
test_model5(df,'spam_marketing',col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9418124999999999

Log-Loss score of this model is

0.16843035595294542

AUC score of this model is

0.5056553096458688



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9383448370335087

Log-Loss score of this model is

0.17960768839508628

AUC score of this model is

0.502


In [6]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,3000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,2000)
test_model3(df,'spam_marketing',GradientBoostingClassifier,col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9524375

Log-Loss score of this model is

0.1311985309527738

AUC score of this model is

0.6367493773852099



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9483248806821896

Log-Loss score of this model is

0.1447122530948653

AUC score of this model is

0.6037573285835799


In [7]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,2000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,3000)
test_model3(df,'spam_marketing',GradientBoostingClassifier,col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9515625

Log-Loss score of this model is

0.13005279207541448

AUC score of this model is

0.6264461353102516



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9445820529824005

Log-Loss score of this model is

0.1546564049410908

AUC score of this model is

0.6167330552609573


In [8]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,3000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,3000)
test_model3(df,'spam_marketing',GradientBoostingClassifier,col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9549375000000001

Log-Loss score of this model is

0.12819821216650434

AUC score of this model is

0.6486578217846427



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9423351733026983

Log-Loss score of this model is

0.16248910604939154

AUC score of this model is

0.5693155846489195


In [9]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)

In [10]:
test_model3(df,'spam_marketing',GradientBoostingClassifier)

TypeError: test_model3() missing 1 required positional argument: 'TFIDF_column'

In [ ]:
y = data_training_set['spam_marketing']
X = data_training_set[['Docusign', 'onespan', 'signnow','adobe sign','listed_count', 'statuses_count','followers_count',
          'favourites_count', 'friends_count','time_float_sin','time_float_cos', 'is_description_none']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

model = RandomForestClassifier()
kf = KFold(n_splits=5, shuffle=True)

ll_performance = []
model_weights = []
#kfold split on X_ (which is X_train of len 110)
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)
    log_ll = log_loss(y_test, y_pred)
    
    ll_performance.append(log_ll)
    #model_weights.append(model.coef_) 